# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

## Multi-Agent Solution

- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

## Modifications of Deep Deterministic Policy Gradients (DDPG) notebook
---


### 1. Import the Necessary Packages

In [1]:
import random as rand
import math
import torch
import numpy as np
import pandas as pd
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from ddpg_agent import Agent

norm = lambda x: (x - x.mean()) / (x.std()+10e-6)
scale = lambda x: (x - x.min()) / (x.max() - x.min())

<max_time=1000>

### 2. Instantiate the Environment and Agent

In [2]:
#use_modules = {'tensorflow': mlagents.tf_utils,
#               'logging': mlagents_envs.logging_util,
#               'torch': mlagents.torch_utils}

In [2]:
#!pip -q install ./python
from unityagents import UnityEnvironment

# select this option to load version 1 (with a single agent) of the environment
#env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
#env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')
env = UnityEnvironment(file_name='.\data\Reacher_Windows_x86_64\Reacher.exe',
                       docker_training=True,
                       no_graphics=True)

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
state = env_info.vector_observations.squeeze()
state_size = state.shape[-1]
print('There are {} agents. Each observes a state with length: {}'.format(num_agents, state_size))
print('The state for the first agent looks like:\n', np.round(state[0], 3))
print('State shape: ', state.shape)

actions = []

UnityTimeOutException: The Unity environment took too long to respond. Make sure that :
	 The environment does not need user interaction to launch
	 The Academy and the External Brain(s) are attached to objects in the Scene
	 The environment and the Python interface have compatible versions.

In [15]:
#env.close()
#env = UnityEnvironment(file_name='.\data\Reacher_Windows_x86_64\Reacher.exe', no_graphics=True)

OSError: handle is closed

ERROR:root:Exception calling application: [WinError 232] The pipe is being closed
Traceback (most recent call last):
  File "C:\Users\David\.conda\envs\drlnd\lib\site-packages\grpc\_server.py", line 385, in _call_behavior
    return behavior(argument, context), True
  File "C:\Users\David\.conda\envs\drlnd\lib\site-packages\unityagents\rpc_communicator.py", line 25, in Exchange
    self.child_conn.send(request)
  File "C:\Users\David\.conda\envs\drlnd\lib\multiprocessing\connection.py", line 206, in send
    self._send_bytes(_ForkingPickler.dumps(obj))
  File "C:\Users\David\.conda\envs\drlnd\lib\multiprocessing\connection.py", line 280, in _send_bytes
    ov, err = _winapi.WriteFile(self._handle, buf, overlapped=True)
BrokenPipeError: [WinError 232] The pipe is being closed


In [12]:
# examine the state space 
state = env_info.vector_observations.squeeze()
state_size = state.shape[-1]
print('There are {} agents. Each observes a state with length: {}'.format(num_agents, state_size))
print('The state for the first agent looks like:\n', [np.round(s, 3) for s in state[0]])
print('State shape: ', state.shape)

There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like:
 [0.0, -4.0, 0.0, 1.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -10.0, 0.0, 1.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.755, -1.0, 5.557, 0.0, 1.0, 0.0, -0.168]
State shape:  (20, 33)


In [11]:
print('avg over agents:\n', [np.round(np.mean(s, axis=0), 3) for s in state])
print('std over agents:\n', [np.round(np.std(s, axis=0), 3) for s in state])
print('max over agents:\n', [np.round(np.max(s, axis=0), 3) for s in state])
print('min over agents:\n', [np.round(np.min(s, axis=0), 3) for s in state])

avg over agents:
 [-0.026, -0.72, -0.095, -0.475, -0.264, -0.646, -0.155, -0.406, -0.45, -0.414, -0.567, -0.148, -0.144, -0.716, -0.159, -0.322, -0.143, -0.42, -0.137, -0.428]
std over agents:
 [2.361, 2.251, 2.36, 2.313, 2.35, 2.272, 2.358, 2.327, 2.319, 2.325, 2.294, 2.357, 2.358, 2.252, 2.361, 2.34, 2.363, 2.324, 2.357, 2.323]
max over agents:
 [5.755, 1.0, 7.902, 3.507, 6.784, 1.0, 7.981, 4.702, 3.756, 4.815, 1.0, 7.956, 7.922, 1.0, 7.762, 6.128, 7.853, 4.474, 7.981, 4.239]
min over agents:
 [-10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0]


In [ ]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
print("state \n", states)
#scores = np.zeros(num_agents)                          # initialize the score (for each agent)
#while True:
actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
print("action \n",actions)
actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
print("action \n",actions)
env_info = env.step(actions)[brain_name]           # send all actions to tne environment
next_states = env_info.vector_observations         # get next state (for each agent)
print("next_state \n", next_states)
rewards = env_info.rewards                         # get reward (for each agent)
print("reward \n",rewards)
dones = env_info.local_done                        # see if episode finished
print("done \n",dones)
#scores += env_info.rewards                         # update the score (for each agent)
#states = next_states                               # roll over states to next time step
#if np.any(dones):                                  # exit loop if episode finished
    #break


#### Sep
___

state = env_info.vector_observations.squeeze()

In [13]:
import numpy as np
import random
import copy
from collections import namedtuple, deque

import torch
import torch.nn.functional as F
import torch.optim as optim

BUFFER_SIZE = int(6.4e5)  # replay buffer size = int(1e6)
BATCH_SIZE = 128        # minibatch size = 128
GAMMA = 0.99          # discount factor = 0.99
TAU = 1e-3              # for soft update of target parameters = 1e-3
LR_ACTOR = 1e-4         # learning rate of the actor  = 1e-4
LR_CRITIC = 1e-3        # learning rate of the critic = 3e-4
WEIGHT_DECAY = 1e-5     # L2 weight decay = 0.0001 to 0.
random_seed = 17
RIGHT = 4 #RIGHT
LEFT = 5 #LEFT

#cunit = torch.cuda.init()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class Agency():
    """Collection of Agent objects who interact with and learn from their environment."""
    
    def __init__(self, num_agents, state_size, action_size, random_seed):
        """Initialize an Agency of Agent objects.      
            Params
            ======
            num_agents (int): number of agents
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            random_seed (int): random seed
        """
        self.num_agents = num_agents
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(random_seed)
        
        agents = []
        for _ in range(num_agents):
            agents.append(Agent(state_size, action_size, seed))
        self.agents = agents
        
    def act(self, states):
        return [agent.act(state) for agent, state in zip(self.agents, states)]
    
    def step(self, states, actions, rewards, next_states, dones):
        for agent, state, action, reward, next_state, done in zip(self.agents, states, actions, rewards, next_states, dones):
            agent.step(state, action, reward, next_state, done)
            
    def watch():
        pass

In [ ]:
seed = 17
from ddpg_agent import Agent
agency = Agency(num_agents, state_size, action_size, random_seed=seed)

### 3. Train the Agents with DDPG

In [ ]:
env_info = env.reset(train_mode=True)[brain_name]
states = env_info.vector_observations
states.shape

In [ ]:
env_info = env.reset(train_mode=True)[brain_name]
states = env_info.vector_observations
#scores = np.zeros(num_agents)
#actions = [agent.act(state) for agent,state in zip(agency.agents, states)]

scores = env_info.rewards
print([a for a in actions])
print(scores)

In [ ]:
def train20(n_episodes=1000, print_every=100, max_score=-np.inf):
    scores_deque = deque(maxlen=100)
    all_scores = []
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        steps = np.zeros(num_agents)
        while True:
            steps += 1
            actions = agency.act(states)                       # list-like of actions (one per agent)
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next states (one per agent)
            rewards = env_info.rewards                         # rewards returned (for each agent)
            dones = env_info.local_done                        # see if any episodes are finished
            agency.step(steps, states, actions, rewards, next_states, dones)
            states = next_states                               # roll over states to next time step
            scores += rewards                                  # update the score (for each agent)
            if np.any(dones):                                  # exit loop if any episode finished
                break    
        scores_deque.append(scores)
        all_scores.append(scores) 
        #if max(all_scores).any() > max_score: max_score = max(all_scores).any()
        epi_max = max([[max(s) for s in scores] for scores in all_scores])
        if max_score < epi_max: max_score = epi_max
        avg_score = np.mean([[np.mean(s) for s in scores] for scores in scores_deque])
        print('\rEpisode {}\tAverage Score: {:.2f}\tHigh Score: {:.2f}'.format(i_episode, avg_score, max_score), end="")
        if i_episode % print_every == 0:
            agent = random.choice(agency.agents) #[a for a in agency.agents])
            torch.save(agent.actor_local.state_dict(), './data/checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), './data/checkpoint_critic.pth')
            print('\rEpisode {}\tAverage Score: {:.2f}\tHigh Score: {:.2f}\n'.format(i_episode, avg_score, max_score))   

    return all_scores, max_score     

In [ ]:
#scores = ddpg(n_episodes=64, print_every=32)
all_scores, mx = train20(n_episodes=120, print_every=20)
#all_scores += ores
all_scores[-10:], mx

In [ ]:
fig = plt.figure(figsize=(12,6))
#for i, scores in enumerate(all_scores):
i=1
ax = fig.add_subplot(211+i)
plt.plot(np.arange(1, len(all_scores[i])+1), all_scores[i])
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
#player_scores = [[pscr for pscr in ground] for ground in all_scores]

### 4. Watch a Smart Agent!

In [ ]:
agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

env_info = env.reset(train_mode=True)[brain_name]
#state = env.reset()
for t in range(200):
    action = agent.act(state, add_noise=False)
    env.render()
    state, reward, done, _ = env.step(action)
    if done:
        break 

#env.close()

In [ ]:
#### Display from DQN notebook
rewards = []
steps = []
tries = 100
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
img = plt.imshow(env.render())
for j in range(1000):
    action = agent.act(state)
    state, reward, done, trun, info = env.step(action)
    rewards += [reward]
    img.set_data(env.render()) 
    plt.axis('off')
    display.display(plt.gcf())
    display.clear_output(wait=True)
    if done or trun:
        break 

In [ ]:
#env.close()